# BalanseRegnskap og regionsaggregering
### Laster inn nødvendige pakker

In [ ]:
import pandas as pd
import numpy as np
exec(open('../Funksjoner m.m./funksjoner.py').read())

## Produserer innputfilene til BalanseRegnskap
Før dette gjøres må følgende filer være kjørt:

1. API Klass
    - Denne henter lister over følgende fra Klass: kapitler, kommuner, fylkeskommuner, kommunehierarki, fylkeskommunehierarki og bydeler
2. Import fra Driller
    - Denne henter følgende skjemaer fra driller: 0B, 0D, 0J, 0L, 0N og 0Q

In [ ]:
%run ./auto_balanse/'Kasse Balanse'.ipynb

In [ ]:
%run ./auto_balanse/'Konsolidert Balanse'.ipynb

In [ ]:
%run ./auto_balanse/'Saerbedriftsbalanse'.ipynb

<br>
<br>

## Kontroller 
### Kasse
**1. Kontrollerer om det er fjernet rader med null i de innleste balansefilene**

In [ ]:
kontroll_merge(bd,bd_null)

<br>

**2. Kontrollerer om det er fjernet rader med ugyldige kapitler**

In [ ]:
kontroll_merge(bd_forts, bd_kapittelsfilter)

<br>

**3. Lister ut manglende kommuner**

In [ ]:
bd_manglende

### Konsolidert

**1. Kontrollerer om det er fjernet rader med null i de innleste balansefilene**

In [ ]:
kontroll_merge(nq,nq_null)

<br>

**2. Kontrollerer om det er fjernet rader med ugyldige kapitler**

In [ ]:
kontroll_merge(nq_forts, nq_kapittelfilter)

<br>

**3. Lister ut manglende kommuner**

In [ ]:
nq_manglende

### Særbedriftene

**1. Kontrollerer om det er fjernet rader med null i de innleste regnskapsfilene**

In [ ]:
kontroll_merge(jl,jl_null)

<br>

**2. Kontrollerer om det er fjernet rader med ugyldige kapitler**

In [ ]:
kontroll_merge(jl_snudd, jl_kapittelfilter)

**3. Kontrollerer eierandelene som er fjernet som følge av at eierkommune ikke har levert balanseregnskapet**

In [ ]:
kontroll_merge(jl_eierandel, jl_eier_levert)

## Henter Hierarkifilene

In [ ]:
kapittelhierarki = pd.read_parquet("/ssb/stamme03/komakro/pilot_python/inndata/" +aar+ "/kapittelhierarki_" +aar+ ".parquet")
kommunehierarki= pd.read_parquet("/ssb/stamme03/komakro/pilot_python/arbeidsdata/klass/" + aar + "/kommunehierarki.parquet")
fylkeskommunehierarki = pd.read_parquet("/ssb/stamme03/komakro/pilot_python/arbeidsdata/klass/" + aar + "/fylkeskommunehierarki.parquet")

In [ ]:
#Siden BalanseRegnskap ikke godtar regnskapsomfang C endret omfang til B som en quick fix
kostra0n_0q_kontrollert['regnskapsomfang'] = kostra0n_0q_kontrollert['regnskapsomfang'].str.replace('C','B')

## BalanseRegnskap

In [ ]:
%load_ext rpy2.ipython

### Leser inn nødvendige pakker

In [ ]:
%R library(Kostra)
%R library(tidyverse)

In [ ]:
%R -i balanse_basis
%R -i kostra0n_0q_kontrollert
%R -i kapittelhierarki
%R -i kommunehierarki
%R -i fylkeskommunehierarki
%R -i bd_manglende

## Balanse regnskapsfunksjonen
- Kjører funksjonen BalanseRegnskap fra Kostra biblioteket. Dette er den samme funksjonen som er lagt inn i Kompis.
- Funksjonen må skrives med inputdata i følgende rekkefølge: 
    1. Balanseregnskapet (Kasse og særbedrifter samlet)
    2. Kapittelhierarki

In [ ]:
%R resultat <- BalanseRegnskap(balanse_basis, kapittelhierarki, omfang = c('A','B','sbedr'))

## Konsern
- Filtrerer ut konsern-tall
- Lager KG33, KG34, KG35, KG36
- Aggregerer region
- Fjerner estimerte kommuner

In [ ]:
%R konsern_start <- filter(resultat, regnskapsomfang == "A")

In [ ]:
%%R ny_var <- resultat %>% 
filter(kapittel %in% c('14','23','33','47') & regnskapsomfang %in% c('B','sbedr')) %>%
pivot_wider(
    names_from = c(regnskapsomfang, kapittel),
    values_from = belop)%>%
mutate (KG33 = B_14 - sbedr_33) %>%
mutate (KG34 = B_23 - sbedr_47) %>%
mutate (KG35 = B_33 - sbedr_14) %>%
mutate (KG36 = B_47 - sbedr_23)

ny_var <- ny_var[c('periode', 'region','KG33','KG34','KG35','KG36')]

ny_var <- ny_var %>% 
pivot_longer(
    cols = starts_with ('KG'), 
    names_to ='kapittel', 
    values_to = 'belop')%>%
add_column(regnskapsomfang = 'A')

In [ ]:
%R konsern <- rbind(konsern_start, ny_var)

### Aggregerer regnskapet
- Kjører funksjon HierarchyCompute fra kostra biblioteket. 
- Funksjonen aggregerer regnskapet til følgende (så sant regionsfierarkiet er korrekt):
    - EAK (Landet - kommuner)
    - EAKUO (Landet uten Oslo - kommuner)
    - EKAXX (Fylker)
    - EKG (kostragrupper)
    - EAFK (Landet - fylkeskommuner)
    - EAFKUO (Landet uten Oslo - fylkeskommuner)
    - EAFKXX (Landsdeler)
<br>
<br>
- Vi lager kun aggregerte størrelser på konserntall. 

In [ ]:
%R regionshierarki <- rbind(kommunehierarki, fylkeskommunehierarki)

In [ ]:
%%R bal_konsern <- HierarchyCompute(data=konsern, valueVar = "belop", 
                      hierarchies=list(region = regionshierarki, regnskapsomfang = "rowFactor",  kapittel = "rowFactor"), 
                      hierarchyVarNames = c(mapsFrom = "from", mapsTo = "to", sign = "sign"), 
                      colVar = "region", rowSelect = unique(konsern[, c("regnskapsomfang", "kapittel")]),
                      inputInOutput = TRUE)

In [ ]:
%%R bal_konsern <- bal_konsern[!bal_konsern$region %in%
                               bd_manglende$region,]

In [ ]:
%R bal_konsern$periode <- '2021'

## Kasse
- Filtrerer ut kasse-tall
- Fjerner estimerte kommuner

In [ ]:
%R bal_kasse <- filter(resultat, regnskapsomfang == "B")

In [ ]:
%%R bal_kasse <- bal_kasse[!bal_kasse$region %in%
                           bd_manglende$region,]

## Konsolidert
- KostraRegnskaps-funksjonen er ikke laget for konsolidert. 
- Konsolidert skal behandles likt som kasse. Mekker derfor til ved å kalle sette regnskapsomfang B på konsolidert og kjøre KostraRegnskap for kun B. 
- Etter kjøringen endres regnskapsomfang tilbake til C. 

In [ ]:
%R resultat_konsolidert <- BalanseRegnskap(kostra0n_0q_kontrollert, kapittelhierarki, omfang = c('B'))

In [ ]:
%R resultat_konsolidert$regnskapsomfang <- gsub('B','C',resultat_konsolidert$regnskapsomfang)

## Samler kasse og konsolidert

In [ ]:
%R bal_kasse_konsolidert <- rbind(bal_kasse, resultat_konsolidert)

## Skriver tilbake til python

In [ ]:
%R -o bal_kasse_konsolidert
%R -o bal_konsern

## Kjører utregning av finansielle grunnlagsdata inkludert rentekompensasjon

In [ ]:
%run ./auto_balanse/'Rentekomp'.ipynb

In [ ]:
fingr

In [ ]:
fingr.to_parquet('/ssb/stamme03/komakro/pilot_python/arbeidsdata/arbeidsfiler/'+aar+'/finansielt_grunnlag_ink_rente.parquet')

In [ ]:
bal_kasse_konsolidert.to_parquet('/ssb/stamme03/komakro/pilot_python/arbeidsdata/arbeidsfiler/'+aar+'/bal_kasse_konsolidert.parquet')